In [ ]:
import cv2
import rosbag
import pandas as pd
import numpy as np
import re
from os import listdir
from os.path import isfile, join

In [ ]:
bag = rosbag.Bag('../bags/trackMe.bag')

In [ ]:
print bag

---
Need to match a frame for each not empty detection msg <br />
How to store this matching? Put everything in CSV format. <br />
So need to create 2 dataframes: one for detection and one for frame <br />
From detection dataframe, need create a none empty detection dataframe (one column "time" and one column "msg")


---

In [ ]:
# open the detection text file
detectionFile = open('/home/sabrine/notebook/messages/detectMeL.txt', 'r') 

In [ ]:
# Create a array with all the messages
detection = detectionFile.read().split('header: \n')[1:]

In [ ]:
# Extract all the information for each messages
seq = []
secs = []
nsecs = []
frame_id = []
detections = []
intrinsic_matrix = []
image_type = []
for i in range(len(detection)):
    seq.append(re.search('[0-9]+',re.search('seq: [0-9]+', detection[i]).group(0)).group(0))
    secs.append(re.search('[0-9]+',re.search('stamp: \n    secs: [0-9]+', detection[i]).group(0)).group(0))
    nsecs.append(re.search('[0-9]+',re.search('nsecs:\s+[0-9]+', detection[i]).group(0)).group(0))
    frame_id.append(re.search('/[A-z]+' ,re.search('frame_id: /[A-z]+', detection[i]).group()).group())
    detections.append(re.search('detections: (?s).*\intrinsic_matrix', detection[i]).group().replace('detections: ', '').replace('intrinsic_matrix', ''))
    intrinsic_matrix.append(re.search('intrinsic_matrix: \[.*\]', detection[i]).group().replace('intrinsic_matrix: ', ''))

In [ ]:
# Create an empty df with the right columns
det = pd.DataFrame(columns=['seq', 'secs', 'nsecs', 'frame_id', 'detections', 'intrinsic_matrix'])

In [ ]:
# Then fill in the empty df with the info extracted before
det.seq = seq
det.secs = secs
det.nsecs = nsecs
det.frame_id = frame_id
det.detections = detections
det.intrinsic_matrix = intrinsic_matrix

In [ ]:
# check the result
det.head()

In [ ]:
det.nsecs()

In [ ]:
# Create a new df with only the none empty detection field
det2 = det[det['detections'].map(len) > 5]

In [ ]:
#Check result
det2.head()

---
Now need to order the detection messages df by secs and then nsces

In [ ]:
# Order the detection messages by secs and nsecs in a new df
det3 = det2.sort_values(by=['secs', 'nsecs'])
det3.index = range(0,len(det3))

In [ ]:
# check result (see 2 last lines of the head)
det3.head(-20)

Now need to create an array of time of frame creation <br />
Then need to math each detection message to a frame by selecting the closest one but younger.

In [ ]:
# List all file from frames folder
mypath = "/home/sabrine/notebook/framesRGB"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
# Extract time from files' name
time_ = []
for t in onlyfiles:
    time_.append(int(re.search('^[0-9]*', t).group()))

In [ ]:
# Sort the list of time in order help the future match
sorted_ = sorted(time_, key=int) 

In [ ]:
# Recreate the time of the detection in order to compare it to the frames' file name
timeDetec = (det3.secs + det3.nsecs).apply(pd.to_numeric).values.T.tolist()

In [ ]:
# Use the fact that the frames'time list is ordered to easly find the closest match with a detection
def closestYounger(detectionTime, list_frameTime):
    closestTime = "error"
    index = "none"
    for i, frameTime in enumerate(list_frameTime):
        if frameTime < detectionTime and (detectionTime-frameTime) < 10000000000:
            closestTime = frameTime
            index = i
        else:
            break
    return index, closestTime

In [ ]:
def synchronizer(detectionList, frameList, dictionary):
    if len(frameList) != 0 and len(detectionList) != 0:
        index, time = closestYounger(detectionList[0], frameList)
        dictionary[detectionList[0]] = time
        print("detection time : ", detectionList[0])
        print("time matched : ", time)
        #print("fameList : ", frameList)
        if len(detectionList) > 0 and len(frameList) > 0:
            print("detection : ", len(detectionList))
            print("frame : ", len(frameList))
            if index != 'none':
                shorterFrameTimeList = frameList[:index] + frameList[index+1 :]
                synchronizer(detectionList[1:], shorterFrameTimeList, dictionary)
            else:
                synchronizer(detectionList[1:], frameList, dictionary)                
    return dictionary

In [ ]:
len(sorted_)

In [ ]:
synchronizer(timeDetec, sorted_, {})

In [ ]:
1491825301642750000-1491825301551694486